## Librerias

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
from PIL import Image
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer,normalize,RobustScaler,MaxAbsScaler,QuantileTransformer,PolynomialFeatures
import warnings
from sklearn.metrics import root_mean_squared_error
warnings.filterwarnings('ignore')

## Data

In [12]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("Data/train_clean.csv")


## Data exploration

In [13]:
train.head()

,laptop_ID,TypeName,Inches,Touchscreen,ProcessorType,ModelIdentifier,Core,ProcessorSpeed(GHz),Ram(GB),Memory SSD GB,...,"PanelType_IPS Panel, Quad HD+","PanelType_IPS Panel, Retina Display",PanelType_No penal type,PanelType_Quad HD+,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS,OpSys_No OS,OpSys_Windows
0,599,1,11.6,1,9.0,730,0.0,1.0,4,128.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,92,2,15.6,0,10.0,9830,0.0,3.0,8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,744,3,15.6,0,1.0,7700,0.0,2.8,16,256.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,729,1,13.3,1,5.0,7200,0.0,2.5,8,256.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,957,4,15.6,0,5.0,6200,0.0,2.3,8,256.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Data processing

In [14]:
df = train.copy()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 55 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   laptop_ID                            912 non-null    int64  
 1   TypeName                             912 non-null    int64  
 2   Inches                               912 non-null    float64
 3   Touchscreen                          912 non-null    int64  
 4   ProcessorType                        911 non-null    float64
 5   ModelIdentifier                      912 non-null    int64  
 6   Core                                 912 non-null    float64
 7   ProcessorSpeed(GHz)                  912 non-null    float64
 8   Ram(GB)                              912 non-null    int64  
 9   Memory SSD GB                        912 non-null    float64
 10  Memory HDD GB                        912 non-null    float64
 11  Memory FL STO GB                

In [16]:
df['ProcessorType'] = df['ProcessorType'].fillna(15)

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [17]:
import numpy as np

In [18]:
# X = df.drop(['Price_euros'], axis=1)
# y = df['Price_euros'].copy()
X = df.drop(['Price_euros'], axis=1)
y = df['Price_euros'].values.reshape(-1, 1)

# poly = PolynomialFeatures(degree=2, include_bias=False)
# X_poly = poly.fit_transform(X)
scaler_y = np.sqrt(y)

### 2. Dividir X_train, X_test, y_train, y_test

In [19]:
scaler_X = RobustScaler()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, scaler_y, test_size = 0.20, random_state = 42)

In [21]:
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

In [28]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [29]:
y_pred = knn.predict(X_test)
np.square(y_pred)

array([[1040.48475865],
       [ 474.12769534],
       [ 810.23067051],
       [ 385.73494389],
       [ 641.64104293],
       [1177.02394739],
       [ 807.71400623],
       [ 927.61215642],
       [1383.21448512],
       [1915.68019857],
       [ 552.07258619],
       [ 415.371931  ],
       [ 338.09890245],
       [1252.98999349],
       [1039.92149278],
       [ 387.05454223],
       [ 676.94252016],
       [ 686.56339195],
       [ 684.99875687],
       [1271.91651172],
       [ 279.52199481],
       [ 780.75920692],
       [2080.73562761],
       [ 467.05824271],
       [ 259.79003847],
       [ 841.36948509],
       [ 421.07327186],
       [2357.63712243],
       [1402.08905811],
       [1215.00450555],
       [1531.50903056],
       [ 841.36948509],
       [1003.88124855],
       [ 679.91360884],
       [ 869.15648816],
       [1000.05589445],
       [2347.69916319],
       [1032.32236604],
       [ 256.09223671],
       [1096.60620786],
       [1379.16910719],
       [1098.862

### 3. Asignar el modelo (vacío) a una variable


### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de RMSE.

In [ ]:
rf = RandomForestRegressor(n_estimators=300, random_state=42)
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Optimización con RandomizedSearchCV
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                   n_iter=20, cv=5, scoring='neg_root_mean_squared_error',
                                   n_jobs=-1, verbose=2, random_state=42)

random_search.fit(X_train, y_train.ravel())

best_rf = random_search.best_estimator_

# Seleccionar las características más importantes
selector = SelectFromModel(best_rf, threshold="median", prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

scaler_X = RobustScaler()
X_train_scaled = scaler_X.fit_transform(X_train_selected)
X_test_scaled = scaler_X.transform(X_test_selected)

rf_selected = RandomForestRegressor(**random_search.best_params_,random_state=42)
rf_selected.fit(X_train_scaled, y_train.ravel())

y_pred_rf = rf_selected.predict(X_test_selected)

# Calcular RMSE
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)
print(f"✅ RMSE Random Forest: {rmse_rf:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


✅ RMSE Random Forest: 32.4238


### 2. Carga los datos de `test.csv` para predecir.

In [30]:
X_pred = pd.read_csv("Data/test_clean.csv")
# X_pred = X_pred.drop(columns=variables_insignificantes)

In [32]:
X_train_poly_df = pd.DataFrame(X_train, columns=poly.get_feature_names_out())

In [33]:
X_pred = X_pred[[col for col in X_train_poly_df.columns if col in X_pred.columns]]

In [34]:
X_poly_pred = poly.transform(X_pred)

In [35]:
X_pred_selected = selector.transform(X_pred)

X_poly_pred_scaled = scaler_X.transform(X_pred_selected) 

predictions_scaled = rf_selected.predict(X_poly_pred_scaled)

# Si aplicaste sqrt() en y, deshacerlo con np.square()
predictions_submit = np.square(predictions_scaled)

# Mostrar algunas predicciones
predictions_submit

array([1269.41051635, 1215.57143846,  258.54995511, 1859.18128249,
       1572.7810275 ,  372.41827132, 2569.61949937,  470.63925847,
       1750.93460376,  798.01991871, 1166.7299905 , 2587.97203936,
        515.59868615,  472.52074178, 1509.49992311, 2237.02081676,
        206.37599168,  889.93660061, 2491.60544507,  453.70884714,
       1511.46159262, 1315.69424618, 1633.42525163,  601.01388985,
        698.90298908,  649.82001443,  675.6070887 , 2044.505452  ,
       1060.23146628,  832.55836294, 1942.27925149, 1393.87635502,
        333.56190421, 1766.65565257, 1054.82077505,  468.18402856,
       1462.96983577, 1531.4469204 ,  770.87002857, 1546.15424466,
       1765.88492022, 1647.34442575, 1789.53150249, 1309.64035608,
       1369.51964616,  767.79461603,  418.32684588, 4456.05604235,
        824.45038009,  675.25555196,  535.97022037,  680.90612755,
        551.51227295,  463.81233422,  934.14469401,  280.12064992,
       1220.17268251,  933.1154379 , 2099.69969538,  917.62526

In [36]:
# X_pred = X_pred[['laptop_ID', 'TypeName', 'Inches', 'ProcessorType', 'ModelIdentifier',
#        'ProcessorSpeed(GHz)', 'Ram(GB)', 'Memory SSD GB', 'Memory HDD GB',
#        'GpuBrand', 'GpuModel', 'Weight(KG)', 'Ancho', 'Alto', 'Pixeles',
#        'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Chuwi',
#        'Company_Dell', 'Company_Fujitsu', 'Company_Google', 'Company_HP',
#        'Company_Huawei', 'Company_LG', 'Company_Lenovo', 'Company_MSI',
#        'Company_Mediacom', 'Company_Microsoft', 'Company_Razer',
#        'Company_Samsung', 'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi',
#        'PanelType_4K Ultra HD', 'PanelType_Full HD', 'PanelType_IPS Panel',
#        'PanelType_IPS Panel, 4K Ultra HD', 'PanelType_IPS Panel, Full HD',
#        'PanelType_IPS Panel, Quad HD+', 'PanelType_IPS Panel, Retina Display',
#        'PanelType_No penal type', 'PanelType_Quad HD+', 'OpSys_Android',
#        'OpSys_Chrome OS', 'OpSys_Linux', 'OpSys_Mac OS', 'OpSys_No OS',
#        'OpSys_Windows']]
# # X_pred = X_pred[['laptop_ID', 'TypeName', 'Inches', 'PanelType', 'Touchscreen',
# #        'ProcessorType', 'ModelIdentifier', 'Core', 'ProcessorSpeed(GHz)',
# #        'Ram(GB)', 'Memory SSD GB', 'Memory HDD GB', 'Memory FL STO GB',
# #        'Memory Hybrid GB', 'GpuBrand', 'GpuModel', 'OpSys', 'Weight(KG)', 'Ancho', 'Alto', 'Pixeles', 'AspectRatio',
# #        'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Dell',
# #        'Company_HP', 'Company_Lenovo', 'Company_MSI', 'Company_Otros',
# #        'Company_Toshiba']]
# X_pred = X_pred[['laptop_ID', 'TypeName', 'Inches', 'ProcessorType', 'ModelIdentifier',
#        'ProcessorSpeed(GHz)', 'Ram(GB)', 'Memory SSD GB', 'Memory HDD GB',
#        'GpuBrand', 'GpuModel', 'Weight(KG)', 'Ancho', 'Alto', 'Pixeles',
#        'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Chuwi',
#        'Company_Dell', 'Company_Fujitsu', 'Company_Google', 'Company_HP',
#        'Company_Huawei', 'Company_LG', 'Company_Lenovo', 'Company_MSI',
#        'Company_Mediacom', 'Company_Microsoft', 'Company_Razer',
#        'Company_Samsung', 'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi',
#        'PanelType_4K Ultra HD', 'PanelType_Full HD', 'PanelType_IPS Panel',
#        'PanelType_IPS Panel, 4K Ultra HD', 'PanelType_IPS Panel, Full HD',
#        'PanelType_IPS Panel, Quad HD+', 'PanelType_IPS Panel, Retina Display',
#        'PanelType_No penal type', 'PanelType_Quad HD+', 'OpSys_Android',
#        'OpSys_Chrome OS', 'OpSys_Linux', 'OpSys_Mac OS', 'OpSys_No OS',
#        'OpSys_Windows']]

In [37]:
predictions_submit

array([1269.41051635, 1215.57143846,  258.54995511, 1859.18128249,
       1572.7810275 ,  372.41827132, 2569.61949937,  470.63925847,
       1750.93460376,  798.01991871, 1166.7299905 , 2587.97203936,
        515.59868615,  472.52074178, 1509.49992311, 2237.02081676,
        206.37599168,  889.93660061, 2491.60544507,  453.70884714,
       1511.46159262, 1315.69424618, 1633.42525163,  601.01388985,
        698.90298908,  649.82001443,  675.6070887 , 2044.505452  ,
       1060.23146628,  832.55836294, 1942.27925149, 1393.87635502,
        333.56190421, 1766.65565257, 1054.82077505,  468.18402856,
       1462.96983577, 1531.4469204 ,  770.87002857, 1546.15424466,
       1765.88492022, 1647.34442575, 1789.53150249, 1309.64035608,
       1369.51964616,  767.79461603,  418.32684588, 4456.05604235,
        824.45038009,  675.25555196,  535.97022037,  680.90612755,
        551.51227295,  463.81233422,  934.14469401,  280.12064992,
       1220.17268251,  933.1154379 , 2099.69969538,  917.62526

In [38]:
sample = pd.read_csv("Data/sample_submission.csv")

In [39]:
sample.head()

,laptop_ID,Price_euros
0,539,650
1,327,650
2,563,650
3,13,650
4,935,500


### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [40]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit.reshape(-1)})

In [41]:
submission.head()

,laptop_ID,Price_euros
0,539,1269.410516
1,327,1215.571438
2,563,258.549955
3,13,1859.181282
4,935,1572.781028


### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [42]:
def chequeator(df_to_submit,version):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit['laptop_ID'].all() == sample['laptop_ID'].all():
                print("You're ready to submit!")
                submission.to_csv(f"submission_{version}.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [43]:
chequeator(submission,9)

You're ready to submit!
